erefore we set the max_features parameter to 1500, which means that we want to use 1500 most occurring words as features for training our classifier.# Lab Week 6
One issue in natural language processing is the identification of sentiment of texts, are they positive, negative or neutral to some concept. This dataset (https://github.com/thanhtut/info284_lab/blob/master/assignment1/twitter-airline-sentiment/Tweets.csv) has a collection of tweets about airlines with corresponding sentiments (negative, positive, neutral).

The task :
The task we want you to do is to run three machine learning models using Naïve Bayes on the tweet texts.

1. Prepare the data for machine learning. Extract the relevant columns, do language preprocessing of the tweets, etc.
2. Run a BernoulliNB to classify sentiment
3. Run a MultiomialNB to classify sentiment
4. Run a MultinomialNB to classify sentiment, but insteadof using a count vector you should use a TF-IDF-vector for each instance. What is TF-IDF? Read sources on information retrieval, and also look here: https://stackabuse.com/text-classification-with-python-and-scikit-learn

How does these three approaches compare?

### What can we tell about the data?
The dtypes vary, none of the data has nulls, we´ll double check anyways, mostly objects witch will be timeforamts and strings.

We can analyse the data:

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("dataset/Tweets.csv")

In [3]:
print("Shape of the dataframe is",df.shape)

Shape of the dataframe is (14640, 15)


In [4]:
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [5]:
print("Percentage null or na values in tweets")
((df.isnull() | df.isna()).sum() * 100 / df.index.size).round(2)

Percentage null or na values in tweets


tweet_id                         0.00
airline_sentiment                0.00
airline_sentiment_confidence     0.00
negativereason                  37.31
negativereason_confidence       28.13
airline                          0.00
airline_sentiment_gold          99.73
name                             0.00
negativereason_gold             99.78
retweet_count                    0.00
text                             0.00
tweet_coord                     93.04
tweet_created                    0.00
tweet_location                  32.33
user_timezone                   32.92
dtype: float64

In [6]:
#print(df.duplicated().any())
#print(len(df))

In [7]:
#df = df.drop_duplicates()
#print(len(df))

### Choosing the necesarry columns:
After removing duplications and looking for emtpy columns we can pick out the colums that should be used.
`airline_sentiment_gold` , `negativereason_gold` and `tweet_coord` is all but empty so they will prove no help.
`tweet_id`, `name` and `tweet_created` are rather unique so they will not give anything to the table. The confidence columns are also uneccesary. That leaves us with the remaining:


In [8]:
tweets = df.drop(columns = [
    "airline_sentiment", "airline_sentiment_confidence", "negativereason_confidence",
    "tweet_id", "airline_sentiment_gold", "name", "negativereason_gold",
    "tweet_coord", "tweet_created", "tweet_location", "user_timezone"
])

In [9]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   negativereason  9178 non-null   object
 1   airline         14640 non-null  object
 2   retweet_count   14640 non-null  int64 
 3   text            14640 non-null  object
dtypes: int64(1), object(3)
memory usage: 457.6+ KB


In [10]:
X = tweets["text"]
y = df["airline_sentiment"]

In [11]:
documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [12]:
print(X.shape)
print(y.shape)
#y = y.to_numpy()
#y = y.reshape(y.shape[0],1)
print("-------------")
print(X.shape)
print(y.shape)

X = documents

(14640,)
(14640,)
-------------
(14640,)
(14640,)


Lets Vectorize the data using `CountVectorizer` the sklearn function converts a collection of text documents to a matrix of token counts

In [13]:
vect = CountVectorizer(max_features = 2000, min_df=5, max_df=0.7, stop_words = stopwords.words("english"))

X = vect.fit_transform(X)

Lets split the data into random train and test subsets and fit it to the `BernoulliNB()` model

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [15]:
model = BernoulliNB().fit(X_train, y_train)
p_train = model.predict(X_train)
p_test = model.predict(X_test)

Now let us calculate the accurasy and look at the results

In [16]:
acc_train = accuracy_score(y_train, p_train)
acc_test = accuracy_score(y_test, p_test)
print(f"Train Accuracy: {acc_train}\n Test Accuracy: {acc_test}")

Train Accuracy: 0.8141165755919855
 Test Accuracy: 0.773224043715847


In [17]:
model = MultinomialNB().fit(X_train, y_train)
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [18]:
acc_train = accuracy_score(y_train, p_train)
acc_test = accuracy_score(y_test, p_test)
print(f"Train Accuracy: {acc_train}\n Test Accuracy: {acc_test}")

Train Accuracy: 0.8103825136612022
 Test Accuracy: 0.771311475409836


> Finding TFIDF

In [19]:
tfidconverter = TfidfTransformer()
X = tfidconverter.fit_transform(X).toarray()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [21]:
model = MultinomialNB().fit(X_train, y_train)
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [22]:
acc_train = accuracy_score(y_train, p_train)
acc_test = accuracy_score(y_test, p_test)
print(f"Train Accuracy: {acc_train}\n Test Accuracy: {acc_test}")

Train Accuracy: 0.7826047358834244
 Test Accuracy: 0.7543715846994535
